In [1]:
from pathlib import Path
import re
import numpy as np
import pandas as pd

DERIV_ROOT = Path("/media/neel/MOUS1/MOUS/MOUS/fmriprep_fresh")  # fMRIPrep derivatives
SUB = "sub-V1039"
TASK = "visual"        # 'visual' in your filenames
TR = 1.2               # set your TR (s)

func_dir = DERIV_ROOT / SUB / "func"

# Match any MNI space, unsmoothed, desc-preproc bold
func_pattern = f"{SUB}_task-{TASK}_space-*desc-preproc_bold.nii*"
func_candidates = sorted(
    p for p in func_dir.glob(func_pattern)
    if not re.match(r"^[Gs]sub-", p.name)  # drop Gsub/ssub smoothed copies
)

if not func_candidates:
    print("No files matched pattern:", func_pattern)
    print("Available func files in", func_dir, ":")
    for p in sorted(func_dir.glob("*bold*.nii*")):
        print("  ", p.name)
    raise FileNotFoundError("No matching desc-preproc BOLD found for that task.")

FUNC = func_candidates[0]
print("Using BOLD file:", FUNC.name)

# Optional: brain mask in same space (not strictly needed for this gPPI)
mask_pattern = FUNC.name.replace("desc-preproc_bold", "desc-brain_mask").rsplit(".", 1)[0] + ".nii.gz"
mask_candidates = sorted(func_dir.glob(mask_pattern))
BOLDMASK = mask_candidates[0] if mask_candidates else None
print("Mask:", BOLDMASK.name if BOLDMASK else None)

Using BOLD file: sub-V1039_task-visual_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii
Mask: sub-V1039_task-visual_space-MNI152NLin6Asym_res-2_desc-brain_mask.nii.gz


In [2]:
import numpy as np
import pandas as pd

# Confounds TSV that actually exists in your func dir
confounds_tsv = func_dir / f"{SUB}_task-{TASK}_desc-confounds_regressors.tsv"
print("Confounds TSV:", confounds_tsv)
assert confounds_tsv.exists(), "Confounds TSV not found."

conf_df = pd.read_csv(confounds_tsv, sep="\t")

# Inspect columns once to verify naming
print("Confound columns (truncated):")
print(conf_df.columns.tolist()[:30])

# --- choose confound regressors (fMRIPrep-style) ---

# 6 motion parameters (translations + rotations)
motion_cols = [c for c in conf_df.columns
               if c.startswith("trans_") or c.startswith("rot_")]

# WM/CSF means if present
wm_csf_cols = [c for c in conf_df.columns if c in ("white_matter", "csf")]

# Optional: global signal, if you want it
gsg_cols = [c for c in conf_df.columns if c == "global_signal"]

# High-pass (cosine) terms, if present
cosine_cols = [c for c in conf_df.columns if c.startswith("cosine")]

conf_cols = motion_cols + wm_csf_cols + gsg_cols + cosine_cols
conf_cols = list(dict.fromkeys(conf_cols))  # de-duplicate, preserve order
print("Using confound columns:", conf_cols)

Xc = conf_df[conf_cols].to_numpy()

# --- scrubbing mask from FD and stdDVARS if present ---

fd_col = next((c for c in conf_df.columns if "framewise_displacement" in c), None)
dvars_col = next((c for c in conf_df.columns if "std_dvars" in c or "stdDVARS" in c), None)

if fd_col is None or dvars_col is None:
    print("Warning: FD or stdDVARS not found; no scrubbing will be applied.")
    sample_mask = None
else:
    fd = conf_df[fd_col].to_numpy()
    dvars = conf_df[dvars_col].to_numpy()

    # FD < 0.5 mm and stdDVARS < 3 are kept; NaNs are treated as okay
    keep = np.ones(len(conf_df), dtype=bool)
    keep &= np.logical_or(np.isnan(fd), fd < 0.5)
    keep &= np.logical_or(np.isnan(dvars), dvars < 3.0)

    sample_mask = np.where(keep)[0]

    n_tp_full = len(conf_df)
    kept = len(sample_mask)
    pct_scrubbed = 100 * (1 - kept / n_tp_full)
    print({"n_tp_full": n_tp_full, "kept": kept, "pct_scrubbed": pct_scrubbed})

Confounds TSV: /media/neel/MOUS1/MOUS/MOUS/fmriprep_fresh/sub-V1039/func/sub-V1039_task-visual_desc-confounds_regressors.tsv
Confound columns (truncated):
['csf', 'csf_derivative1', 'csf_derivative1_power2', 'csf_power2', 'white_matter', 'white_matter_derivative1', 'white_matter_derivative1_power2', 'white_matter_power2', 'global_signal', 'global_signal_derivative1', 'global_signal_power2', 'global_signal_derivative1_power2', 'std_dvars', 'dvars', 'framewise_displacement', 'rmsd', 't_comp_cor_00', 't_comp_cor_01', 't_comp_cor_02', 't_comp_cor_03', 'a_comp_cor_00', 'a_comp_cor_01', 'a_comp_cor_02', 'a_comp_cor_03', 'a_comp_cor_04', 'a_comp_cor_05', 'a_comp_cor_06', 'a_comp_cor_07', 'a_comp_cor_08', 'a_comp_cor_09']
Using confound columns: ['trans_x', 'trans_x_derivative1', 'trans_x_derivative1_power2', 'trans_x_power2', 'trans_y', 'trans_y_derivative1', 'trans_y_derivative1_power2', 'trans_y_power2', 'trans_z', 'trans_z_derivative1', 'trans_z_derivative1_power2', 'trans_z_power2', 'rot_

In [ ]:
from nilearn import datasets
from nilearn.maskers import NiftiMapsMasker

# Fetch a ready-made network atlas (probabilistic maps)
atlas = datasets.fetch_atlas_msdl()
maps_img = atlas.maps      # 4D probabilistic maps
labels = atlas.labels      # list of ROI labels

# Use NiftiMapsMasker for probabilistic maps
masker = NiftiMapsMasker(
    maps_img=maps_img,
    standardize="zscore_sample",
    detrend=False,
    resampling_target="maps",
)

time_series = masker.fit_transform(
    str(FUNC),
    confounds=Xc,
    sample_mask=sample_mask,  # can be None if you did not define a mask
)

print("Time series shape (n_timepoints x n_rois):", time_series.shape)
print("Number of ROIs:", len(labels))

[fetch_atlas_msdl] Dataset found in /home/neel/nilearn_data/msdl_atlas
